> **음원 데이터 얻기**

melon 사이트에서 수집한 top 100 music에 대한 meta 데이터를 활용하여 `{가수}  {노래 제목}`으로 유튜브에 검색 후 가장 상단의 유튜브 영상에서 유튜브 음원을 추출 후 저장하는 노트북

`프로젝트 기간이 종료된 후 작동하지 않을 수 있음`

In [1]:
#@title 데이터 및 패키지 다운로드
#@markdown meta data와 음성 데이터를 다루기 위한 패키지를 다운로드 합니다.

# meta data download
!wget -q https://raw.githubusercontent.com/kangmg/image2music/main/backend/data_crawling/top_music_meta.csv -O ./top_music_meta.csv

# package downlaod
!pip install -q pytube
!pip install -q ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 584.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [20]:
#@title setting
from pytube import Search
from pytube import YouTube
import time
import os
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from scipy.io import wavfile
from IPython.display import clear_output
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_audio
from scipy.io.wavfile import write
import glob
import shutil
import matplotlib.pyplot as plt
import subprocess
import warnings
warnings.filterwarnings("ignore")

def convertor(path, id, sampling_rate=22050):
    '''
    mp4 -> wav (first 120 seconds)
    Ensure the audio has a consistent length of sampling_rate * 120 samples.
    sampling_rate: sampling rate for the output audio file (default is 22050 Hz)
    '''
    input_path = os.path.join(path, f"{id}.mp4")
    output_wav_path = os.path.join(path, f"{id}.wav")
    temp_output_path = os.path.join(path, f"{id}_temp.wav")

    # Extract audio from the video using ffmpeg directly
    ffmpeg_extract_audio(input_path, temp_output_path, fps=sampling_rate, )
    time.sleep(0.1)

    # Load the audio data from the temporary file
    sr, audio_array = wavfile.read(temp_output_path)

    # Ensure the audio has a consistent length of ( sampling_rate * 120 ) samples
    target_length = int(sampling_rate * 120)

    # Flatten to mono if stereo
    if audio_array.ndim > 1:
        audio_array = audio_array.mean(axis=1)

    # ensure 120 sec
    if len(audio_array) > target_length:
        audio_array = audio_array[:target_length]
    else:
        time.sleep(0.1)
        os.remove(temp_output_path)
        os.remove(input_path)
        time.sleep(0.1)
        raise Exception("Audio length is shorter than 120 seconds.")

    audio_array = audio_array[:target_length]

    # Save the adjusted audio data to the output path
    write(output_wav_path, sampling_rate, audio_array)

    # Remove the temporary and original MP4 files
    time.sleep(0.1)
    os.remove(temp_output_path)
    os.remove(input_path)
    time.sleep(0.1)

"""
def query2download(yt_query: str, download_path:str="/content/audio/", sr=22050, music_len_sec=120)->None:
  '''
    youtube에서 qurey를 검색하고 젤 상단의 영상 오디오 데이터를 wav 형태로 다운받는 함수

    - yt_query: 검색어
    - download_path: 다운받을 경로 ( 파일 이름은 yt id )
    - sr : sampling rate
    - music_len_sec : 음원의 몇초로 자를 것인지 ( default 120 sec )
  '''
  # search
  search = Search(yt_query)
  first_content_link = search.results[0].watch_url

  # download music from youtube
  yt = Youtube(first_content_link)
  yt_id = yt.video_id

  # check length > 2 min
  if yt.length < music_len_sec:
    raise Exception(f"[Warning] ",yt_id, " : 영상 다운 실패")

  # get yt audio file
  yt_id = yt.video_id
  audio = yt.streams.filter(only_audio=True, audio_codec="mp4a.40.5" if True else "mp4a.40.2")[0] # 변경 필요
  name = yt_id + ".mp4"
  try:
    audio.download(output_path=download_path, filename=name)
    time.sleep(0.2)
    convertor(download_path, yt_id,  sampling_rate=sr)
  except Exception as E:
    print("[Warning] ",audio.video_id, " : 영상 다운 실패")
"""

def downUrl(url:str,download_path="/content/audio/", sr=22050)->None:
  '''
  url에 해당하는 유튜브 영상을 오디오 형식으로 저장
  '''
  yt = YouTube(url)
  yt_id = yt.video_id
  #audio = yt.streams.filter(only_audio=True, audio_codec="mp4a.40.5" if True else "mp4a.40.2")[0] # 변경 필요
  audio = yt.streams.filter(only_audio=True).first()
  name = yt_id + ".mp4"
  try:
    audio.download(output_path=download_path, filename=name)
    time.sleep(0.2)
    convertor(download_path, yt_id,  sampling_rate=sr)
  except Exception as E:
    print(E)
    print("[Warning] ", yt_id, " : 영상 다운 실패")
"""
def normalize(spec):
    '''
    normalize mel spectrogram array
    '''
    eps=1e-6

    # Z-score normalization
    mean = spec.mean()
    std = spec.std()
    spec_norm = (spec - mean) / (std + eps)
    spec_min, spec_max = spec_norm.min(), spec_norm.max()

    # Min-max scaling
    spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
    spec_scaled = spec_scaled.astype(np.uint8)

    return spec_scaled


def save_mel_spectrogram(wav_file, save_path):
    '''
    Generate and save Mel spectrogram as a NumPy array.

    Parameters:
    wav_file (str): Path to the input WAV file.
    save_path (str): Path to save the Mel spectrogram array.
    '''
    y, sr = librosa.load(wav_file)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    S_dB = librosa.power_to_db(S, ref=np.max)
    S_dB = normalize(S_dB)
    np.save(save_path, S_dB)


def process_wav_files(input_dir, output_dir="auto"):
  '''
  input_dir에 존재한 *.wav 파일을 output_dir에
  mel spectrogram array를 *.npy로 저장
  '''
  if output_dir == "auto":
    output_dir = input_dir + "_mel"
    os.mkdir(output_dir)
  else:
    if not os.path.isdir(output_dir):
      os.mkdir(output_dir)


  files = os.listdir(input_dir)
  for file in tqdm(files):
    if file.endswith('.wav'):
      wav_file = os.path.join(input_dir, file)
      save_mel_spectrogram(wav_file, os.path.join(output_dir, file.replace('.wav', '.npy')))
"""
def compress2targz(folder_path, output_filename='auto'):
    """
    압축하고자 하는 폴더를 tar.gz 형식으로 압축하는 함수
    """
    if output_filename == 'auto':
        output_filename = f"{folder_path}.tar.gz"
    try:
        subprocess.run(['tar', '-czvf', output_filename, folder_path], check=True)
        print(f"Successfully compressed! :  {output_filename}")
    except subprocess.CalledProcessError as e:
        print(f"Fail to compress")

def get_top_yt_url(search_query:str, warning_msg=True):
  '''
  description
    - youtube에서 qurey를 검색하고 가장 상단의 영상 링크를 반환하는 함수
      - 만약 영상의 길이가 10분 이상이면 두번째 영상의 링크를 반환
    - 개인화 추천 시스템으로 인해서 직접 검색했을 때랑 결과가 다를 수 있음

  parameters
    - search_query(str): 검색어
    - warning_msg(bool): 검색 실패 시 경고 메시지 출력 여부

  returns
    - yt_link(str): 가장 상단의 영상 링크
  '''
  # search
  try:
    search = Search(search_query)
    first_content = search.results[0]
    #check
    if first_content.length > 600:
      # 2번째도 10분보다 길면 None 반환 / 3번째 부터는 음원 신뢰도가 떨어진다고 판단
      if search.results[1].length > 600: return None
      yt_link = search.results[1].watch_url
    else:
      yt_link = first_content.watch_url
    clear_output(wait=True)
    return yt_link
  except:
    if warning_msg : print("[Warning] ",search_query, " : 검색 실패")
    return None

def progress_bar(total, current)->None:
  """
  Description
  -----------
    Display a progress bar indicating the completion status of a task.

  Parameters
  ----------
    - total (int): The total number of units representing the task's completion.
    - current (int): The current progress, representing the number of units completed.

  Example
  -------
  >>> progress_bar(100, 50)
  Processing . . .
    50.0%  |===========>                |  50 / 100
  """
  percent = round(current/total * 100, 2)
  num_progress_bar = round(int(percent) // 5)
  num_redidual_bar =  20 - num_progress_bar
  progress_bar_string = "\033[34mProcessing . . .  \n  {}%  |{}>{}|  {} / {}\033[0m".format(percent, num_progress_bar * "=", num_redidual_bar * " ", current, total)
  print(progress_bar_string)


# 다운로드 실패 url 리스트
failed_url = list()

In [ ]:
#@title get youtube url
# top_music = pd.read_csv('./top_music_meta.csv')

# # yt 링크 리스트 얻기
# yt_url_list = list()
# status = 0
# for row in top_music.iterrows():
#   status += 1
#   progress_bar(top_music.shape[0], status)
#   artist = row[1]['artist']
#   title = row[1]['title']
#   search_query = f"{artist} {title}"
#   yt_url_list.append(get_top_yt_url(search_query, warning_msg=True))

# clear_output()
# !echo "Done!"

In [3]:
# top_music["youtube_top_url"] = yt_url_list

In [12]:
# top_music.shape

(6676, 6)

In [13]:
# 결측치 행
#top_music[top_music.isnull().any(axis=1)]

# 데이터상 하자가 조금 있어보임..!
# 오래된 년도 데이터의 경우, 멜론 웹사이상상 데이터도 일부 오류 존재

,image_url,artist,title,top_year,class,youtube_top_url
142,https://cdnimg.melon.co.kr/cm/album/images/009...,이동기,이동기 - 페이지 이동,1984,kpop,None
546,https://cdnimg.melon.co.kr/cm/album/images/010...,신윤식,신윤식 - 페이지 이동,1988,kpop,None
688,https://cdnimg.melon.co.kr/cm/album/images/010...,최준호,최준호 - 페이지 이동,1989,kpop,None
952,https://cdnimg.melon.co.kr/cm/album/images/000...,서정욱,서정욱 - 페이지 이동,1992,kpop,None
1070,https://cdnimg.melon.co.kr/cm/album/images/010...,데이지,데이지 - 페이지 이동,1994,kpop,None
2552,https://cdnimg.melon.co.kr/cm/album/images/000...,플라이 투 더 스카이,Day By Day,2000,kpop,None
5610,https://cdnimg.melon.co.kr/cm/album/images/021...,DJ Chuckie,DJ Chuckie - 페이지 이동,2013,pop,None
5643,https://cdnimg.melon.co.kr/cm/album/images/021...,Jeff Bernat,Moonlight Chemistry,2013,pop,None


In [14]:
# top_music = top_music.dropna()
# #top_music.to_csv('/content/drive/MyDrive/top_music.csv', index=False)
# top_music.shape

(6668, 6)

In [16]:
# top_music["yt_id"] = top_music["youtube_top_url"].apply(lambda url: YouTube(url).video_id)
# top_music.to_csv('/content/drive/MyDrive/top_music.csv', index=False)
# top_music

In [20]:
# top_music.head()

,image_url,artist,title,top_year,class,youtube_top_url,yt_id
0,https://cdnimg.melon.co.kr/cm/album/images/000...,윤시내,고목,1980,kpop,https://youtube.com/watch?v=C-ZWHdSgrdA,C-ZWHdSgrdA
1,https://cdnimg.melon.co.kr/cm/album/images/020...,희자매,그 사람 바보,1980,kpop,https://youtube.com/watch?v=4lEhHkdLl-0,4lEhHkdLl-0
2,https://cdnimg.melon.co.kr/cm/album/images/000...,계은숙,기다리는 여심,1980,kpop,https://youtube.com/watch?v=hpWvqLQSflY,hpWvqLQSflY
3,https://cdnimg.melon.co.kr/cm/album/images/000...,윤항기,나는 행복합니다,1980,kpop,https://youtube.com/watch?v=-6Zd1mhlHRA,-6Zd1mhlHRA
4,https://cdnimg.melon.co.kr/cm/album/images/000...,김학래,내가 (대상),1980,kpop,https://youtube.com/watch?v=ivkRj7NRz8g,ivkRj7NRz8g


In [4]:
top_music = pd.read_csv('/content/drive/MyDrive/top_music.csv')
top_music.head()

,image_url,artist,title,top_year,class,youtube_top_url,yt_id
0,https://cdnimg.melon.co.kr/cm/album/images/000...,윤시내,고목,1980,kpop,https://youtube.com/watch?v=C-ZWHdSgrdA,C-ZWHdSgrdA
1,https://cdnimg.melon.co.kr/cm/album/images/020...,희자매,그 사람 바보,1980,kpop,https://youtube.com/watch?v=4lEhHkdLl-0,4lEhHkdLl-0
2,https://cdnimg.melon.co.kr/cm/album/images/000...,계은숙,기다리는 여심,1980,kpop,https://youtube.com/watch?v=hpWvqLQSflY,hpWvqLQSflY
3,https://cdnimg.melon.co.kr/cm/album/images/000...,윤항기,나는 행복합니다,1980,kpop,https://youtube.com/watch?v=-6Zd1mhlHRA,-6Zd1mhlHRA
4,https://cdnimg.melon.co.kr/cm/album/images/000...,김학래,내가 (대상),1980,kpop,https://youtube.com/watch?v=ivkRj7NRz8g,ivkRj7NRz8g


In [22]:
#@title youtube audio download

path_to_download = "/content/drive/MyDrive/music_data_base"

download_result = list()

for yt_url, yt_id in tqdm(zip(top_music['youtube_top_url'], top_music["yt_id"]), total=top_music.shape[0]):
  try:
    downUrl(url=yt_url, download_path=path_to_download)
    download_result.append(True)
  except Exception as E:
    print(f"{E}\n")
    print("[Warning] Fail to download : ",yt_url)
    failed_url.append(yt_url)
    download_result.append(False)

# 다운로드 성공 여부 / 저장이 안됨..
# top_music["download_status"] = download_result
# top_music.to_csv('/content/drive/MyDrive/top_music_status.csv', index=False)

clear_output()
!echo "Download Done!"

Download Done!


In [2]:
# 파일 압축
%cd /content/drive/MyDrive
!tar -czvf music_data_base.tar.gz music_data_base

clear_output()
!echo "Compression Done!"

Compression Done!


> 파일 다운로드 여부

In [22]:
import pandas as pd
top_music = pd.read_csv("/content/drive/MyDrive/top_music.csv")
top_music.head()

,image_url,artist,title,top_year,class,youtube_top_url,yt_id
0,https://cdnimg.melon.co.kr/cm/album/images/000...,윤시내,고목,1980,kpop,https://youtube.com/watch?v=C-ZWHdSgrdA,C-ZWHdSgrdA
1,https://cdnimg.melon.co.kr/cm/album/images/020...,희자매,그 사람 바보,1980,kpop,https://youtube.com/watch?v=4lEhHkdLl-0,4lEhHkdLl-0
2,https://cdnimg.melon.co.kr/cm/album/images/000...,계은숙,기다리는 여심,1980,kpop,https://youtube.com/watch?v=hpWvqLQSflY,hpWvqLQSflY
3,https://cdnimg.melon.co.kr/cm/album/images/000...,윤항기,나는 행복합니다,1980,kpop,https://youtube.com/watch?v=-6Zd1mhlHRA,-6Zd1mhlHRA
4,https://cdnimg.melon.co.kr/cm/album/images/000...,김학래,내가 (대상),1980,kpop,https://youtube.com/watch?v=ivkRj7NRz8g,ivkRj7NRz8g


In [23]:
import os
music_list = glob.glob('/content/drive/MyDrive/music_data_base/*.wav')
music_id = [os.path.basename(os.path.splitext(music)[0]) for music in music_list]
print(music_id[:3])

['ErntJrtQGBg', 'U7uxf8kBBlg', 'yYedTpODkx8']


In [26]:
def is_downloaded(id):
  if id in music_id:
    return True
  else:
    return False

In [29]:
top_music[ "is_downloaded" ] = top_music["yt_id"].apply(is_downloaded)
top_music.head()

,image_url,artist,title,top_year,class,youtube_top_url,yt_id,is_downloaded
0,https://cdnimg.melon.co.kr/cm/album/images/000...,윤시내,고목,1980,kpop,https://youtube.com/watch?v=C-ZWHdSgrdA,C-ZWHdSgrdA,True
1,https://cdnimg.melon.co.kr/cm/album/images/020...,희자매,그 사람 바보,1980,kpop,https://youtube.com/watch?v=4lEhHkdLl-0,4lEhHkdLl-0,True
2,https://cdnimg.melon.co.kr/cm/album/images/000...,계은숙,기다리는 여심,1980,kpop,https://youtube.com/watch?v=hpWvqLQSflY,hpWvqLQSflY,True
3,https://cdnimg.melon.co.kr/cm/album/images/000...,윤항기,나는 행복합니다,1980,kpop,https://youtube.com/watch?v=-6Zd1mhlHRA,-6Zd1mhlHRA,True
4,https://cdnimg.melon.co.kr/cm/album/images/000...,김학래,내가 (대상),1980,kpop,https://youtube.com/watch?v=ivkRj7NRz8g,ivkRj7NRz8g,True


In [30]:
top_music.to_csv('/content/drive/MyDrive/top_music_status.csv', index=False)

In [31]:
top_music[ top_music["is_downloaded"] == False ]

,image_url,artist,title,top_year,class,youtube_top_url,yt_id,is_downloaded
30,https://cdnimg.melon.co.kr/cm/album/images/000...,송창식,가나다라,1981,kpop,https://youtube.com/watch?v=AtWazGIQCgk,AtWazGIQCgk,False
34,https://cdnimg.melon.co.kr/cm/album/images/000...,김만수,그 사람,1981,kpop,https://youtube.com/watch?v=fWwR3OSP1XY,fWwR3OSP1XY,False
49,https://cdnimg.melon.co.kr/cm/album/images/003...,김현식,봄 여름 가을 겨울,1981,kpop,https://youtube.com/watch?v=j9aGWUFQUFY,j9aGWUFQUFY,False
151,https://cdnimg.melon.co.kr/cm/album/images/000...,한마음,갯바위,1984,kpop,https://youtube.com/watch?v=TVVrmailwsg,TVVrmailwsg,False
152,https://cdnimg.melon.co.kr/cm/album/images/003...,김형룡,사랑의 벼리,1984,kpop,https://youtube.com/watch?v=tOW5DCWsS8A,tOW5DCWsS8A,False
174,https://cdnimg.melon.co.kr/cm/album/images/000...,정수라,아! 대한민국,1984,kpop,https://youtube.com/watch?v=hXBkyaOVgck,hXBkyaOVgck,False
265,https://cdnimg.melon.co.kr/cm/album/images/003...,건아들,금연,1985,kpop,https://youtube.com/watch?v=KKtwSLubpxs,KKtwSLubpxs,False
306,https://cdnimg.melon.co.kr/cm/album/images/000...,윤시내,그대에게서 벗어나고파,1985,kpop,https://youtube.com/watch?v=NEr14e7C5-M,NEr14e7C5-M,False
311,https://cdnimg.melon.co.kr/cm/album/images/000...,한마음,갯바위,1985,kpop,https://youtube.com/watch?v=TVVrmailwsg,TVVrmailwsg,False
316,https://cdnimg.melon.co.kr/cm/album/images/003...,민지,민지 - 페이지 이동,1985,kpop,https://youtube.com/watch?v=jk0U81_sO1U,jk0U81_sO1U,False
